In [17]:
import os 
import json 


import os 
import json 

def load_json_data(json_file):
    """加载JSON数据"""
    try:
        with open(json_file, 'r', encoding='utf-8') as f:
            return json.load(f)
    except Exception as e:
        print(f"加载JSON文件时出错: {e}")
        return {}

json_file="/data/home/libo/work/DataFactory/results/4o_pk_o4_mini_1_stage_merge/component_consistency_results.json"
json_data = load_json_data(json_file)
    

all_data = []
for image_name, data in json_data.items():
    component_details=data["component_details"]
    components = data["components"]
    for component_name, component_data in component_details.items():
        eval_result = component_data["eval_result"]
        try:
            o4_mini_result = component_data.get("o4-mini",{})['description']
            better_model = eval_result.get("right_model", "none")
            score_details= eval_result["score_details"]
            consistency_score= eval_result["consistency_score"]
            all_data.append([image_name,len(components),component_name,o4_mini_result,better_model,consistency_score,score_details])
        except Exception as e:
            print(f"处理组件 {component_name} 时出错: {e}:{eval_result}")
            continue

处理组件 UVD 时出错: 'description':{'component_pair': 'UVD', 'is_consistent': False, 'consistency_score': 0, 'score_details': [0, 4, 0, 0], 'better_model': 'model1', 'reasoning': '由于模型2返回内容为空，无法匹配任何I/O。模型1定义了4个I/O：V_G、V_DD输入，以及V_OUT、V_B输出。匹配数量N_consistent=0，N_total_model1=4，N_total_model2=0，base_score=100×(2×0)/(4+0)=0。根据“比较上下文为空”的边界情况规则，将is_consistent设为false，consistency_score设为0。'}
处理组件 Adder_3 时出错: 'description':{'component_pair': 'Adder_3', 'is_consistent': False, 'consistency_score': 0, 'score_details': [0, 3, 0, 0], 'better_model': 'model1', 'reasoning': '由于Model2分析为空或出错（生成时出错），无法与Model1的3个I/O信息（2个输入i_diff_DC^*、i_diff；1个输出i_diff_va^*）进行匹配。根据Edge Case规则，当任一分析为空时，直接判定不一致，得分0。计分过程：N_consistent=0，N_total_model1=3，N_total_model2=0，base_score=100×2×0/(3+0)=0。'}
处理组件 directional coupler_2 时出错: 'description':{'component_pair': 'directional coupler_2', 'is_consistent': False, 'consistency_score': 0, 'score_details': [0, 3, 0, 0], 'reasoning': '由于 Model2 的分析内容为空，无法比较连接一致性。', 'right_model': 'model1

In [20]:
import pandas as pd
df = pd.DataFrame(all_data, columns=['image_name', 'components', 'component_name', 'o4_mini_result', 'better_model', 'consistency_score', 'score_details'])
df.to_csv('o4_mini_result.csv', index=False)
df.head()


,image_name,components,component_name,o4_mini_result,better_model,consistency_score,score_details
0,block_circuit_train_15k_0321_003773.jpg,6,PFD/CP,## Connections of PFD/CP\n* **Inputs**:\n *...,none,100.0,"[3, 3, 3, 100]"
1,block_circuit_train_15k_0321_003773.jpg,6,OFF-CHIP LF,## Connections of OFF-CHIP LF\n* **Inputs**:\n...,none,100.0,"[2, 2, 2, 100]"
2,block_circuit_train_15k_0321_003773.jpg,6,VCO,## Connections of VCO\n* **Inputs**:\n * OF...,none,50.0,[]
3,block_circuit_train_15k_0321_003773.jpg,6,PROGRAMMABLE M EVEN DIVIDER,## Connections of PROGRAMMABLE M EVEN DIVIDER\...,none,100.0,"[3, 3, 3, 100]"
4,block_circuit_train_15k_0321_003773.jpg,6,PROGRAMMABLE N DIVIDER,```markdown\n##Connections of PROGRAMMABLE N D...,none,50.0,[]


In [ ]:
pd_data = df[df.consistency_score >= 90]
pd_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1662 entries, 0 to 11263
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   image_name         1662 non-null   object 
 1   components         1662 non-null   int64  
 2   component_name     1662 non-null   object 
 3   o4_mini_result     1662 non-null   object 
 4   better_model       1662 non-null   object 
 5   consistency_score  1662 non-null   float64
 6   score_details      1662 non-null   object 
dtypes: float64(1), int64(1), object(5)
memory usage: 103.9+ KB


In [38]:
pd_data = df
pd_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11306 entries, 0 to 11305
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   image_name         11306 non-null  object 
 1   components         11306 non-null  int64  
 2   component_name     11306 non-null  object 
 3   o4_mini_result     11306 non-null  object 
 4   better_model       11306 non-null  object 
 5   consistency_score  11306 non-null  float64
 6   score_details      11306 non-null  object 
dtypes: float64(1), int64(1), object(5)
memory usage: 618.4+ KB


In [40]:
query_labeled_data=[]
query_list=[
    "What are the connections for node {}?",
    "Show me all connections for node {}.",
    "Please retrieve the list of nodes connected to node {}.",
    "I need to find all the relationships associated with node {}.",
    "Could you return a complete list of all adjacent nodes for node {}?",
    "Fetch the full connection data, including types and weights, for the specified node {}.",
    "For the purpose of my analysis, please provide all incoming and outgoing edges for node {}.",
    "Query the graph database and display all entities that have a direct relationship with node {}.",
    "I need to investigate the network structure around node {}; please pull all of its first-degree connections.",
]
import random
data_cls="node_connection"
for i,r in pd_data.iterrows():
    response = r['o4_mini_result']
    image_name = r['image_name']
    component_name = r['component_name']

    query_labeled_data.append({'images':image_name,'query':random.choice(query_list).format(component_name),'response':response,'objects': {},'data_cls':data_cls})
len(query_labeled_data)

11306

In [41]:
save_path= "/data/home/libo/work/data_juice/dataset/train_data/only_node_conection/1-stage_1k"
with open(os.path.join(save_path, '1-stage_1k_node_connection_all.json'), 'w', encoding='utf-8') as f:
    json.dump(query_labeled_data, f, ensure_ascii=False, indent=2)


[{'images': 'block_circuit_train_15k_0321_003773.jpg',
  'query': 'Please retrieve the list of nodes connected to node PFD/CP.',
  'response': '## Connections of PFD/CP\n* **Inputs**:\n    * 1 MHz: a line with an arrow pointing into PFD/CP from the 1 MHz source  \n    * PROGRAMMABLE N DIVIDER: a line with an arrow pointing into PFD/CP from the Programmable N Divider\n* **Outputs**:\n    * OFF-CHIP LF: a line with an arrow pointing away from PFD/CP toward the OFF-CHIP LF\n* **Bidirectional**:\n    * (none)',
  'objects': {},
  'data_cls': 'node_connection'},
 {'images': 'block_circuit_train_15k_0321_003773.jpg',
  'query': 'Show me all connections for node OFF-CHIP LF.',
  'response': '## Connections of OFF-CHIP LF\n* **Inputs**:\n    * PFD/CP: Unlabeled line from PFD/CP into OFF-CHIP LF\n* **Outputs**:\n    * VCO: Line labeled “14 MHz to 28 MHz” from OFF-CHIP LF to VCO\n* **Bidirectional**: none',
  'objects': {},
  'data_cls': 'node_connection'},
 {'images': 'block_circuit_train_15k_0